In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Content
The water_potability.csv file contains water quality metrics for 3276 different water bodies.

1. pH value:
PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.

2. Hardness:
Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

3. Solids (Total dissolved solids - TDS):
Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.

4. Chloramines:
Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.

5. Sulfate:
Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.

6. Conductivity:
Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.

7. Organic_carbon:
Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.

8. Trihalomethanes:
THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.

9. Turbidity:
The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.

10. Potability:
Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import  scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics  import accuracy_score,classification_report,roc_auc_score,plot_roc_curve,plot_precision_recall_curve

In [ ]:
data=pd.read_csv('/kaggle/input/water-potability/water_potability.csv')

In [ ]:
data

### EDA

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
data.isnull().sum()# here we have found some missing value

### UNIVARIATE ANALYSIS

In [ ]:
i=1
plt.figure(figsize=(15,20))
for j in data.columns:
    plt.subplot(4,3,i)
    sns.boxplot(data=data,x=data[j])
    i+=1

#### we can see that each variable contain some outliers

In [ ]:
data.hist(figsize=(10,10))
plt.show()

#### we can see that each variable is almost normally distributed

### MISSING VALUE TREATMENT

In [ ]:
def missing_mean(x):
    x=x.fillna(x.mean())
    return x

In [ ]:
data=data.apply(lambda x:missing_mean(x))

In [ ]:
data.isnull().sum()

#### missing value has been treated 

### OUTLIER TREATMENT

In [ ]:
def outlier_removal(x):
    upper=x.quantile(.99)
    lower=x.quantile(.01)
    x=np.where(x>upper,upper,x)
    x=np.where(x<lower,lower,x)
    return x

In [ ]:
data=data.apply(lambda x:outlier_removal(x))

In [ ]:
i=1
plt.figure(figsize=(10,15))
for j in data.columns:
    plt.subplot(4,3,i)
    sns.boxplot(data=data,x=data[j])
    i+=1

#### outliers has been treated

### BIVARIATE ANALYSIS

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(),linewidths=1,cmap='RdPu_r',annot=True)

In [ ]:
sns.pairplot(data)

In [ ]:
data.corr()['Potability'].plot(kind='barh')

#### we can see that there is no high  correlation exist among independent variables

In [ ]:
f,axis=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['ph'],ax=axis[0])
sns.boxplot(data=data,x='Potability',y='ph',ax=axis[1])
sns.histplot(data=data,x='ph',hue='Potability',ax=axis[2],kde=True)

In [ ]:
ph_0=data.groupby('Potability').get_group(0)[['ph']]
ph_1=data.groupby('Potability').get_group(1)[['ph']]
print(ph_0.mean())
print(ph_1.mean())
stats.ttest_ind(ph_0,ph_1)

#### we can see that ph is not singificantly different for both category(0,1), and its mean ph lies in the safe range prescribed by WHO (ph=6.5-8.5)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Hardness'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Hardness',ax=ax[1])
sns.histplot(data=data,x='Hardness',ax=ax[2],hue='Potability',kde=True)

In [ ]:
hard_0=data.groupby('Potability').get_group(0)[['Hardness']]
hard_1=data.groupby('Potability').get_group(1)[['Hardness']]

In [ ]:
print(data[['Hardness']].mean())
print(hard_0.mean())
print(hard_1.mean())
print(stats.ttest_ind(hard_0,hard_1))

#### we can see that their is no significant difference of harndness in both category of water(0,1).

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Solids'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Solids',ax=ax[1])
sns.histplot(data=data,x='Solids',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Chloramines'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Chloramines',ax=ax[1])
sns.histplot(data=data,x='Chloramines',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Sulfate'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Sulfate',ax=ax[1])
sns.histplot(data=data,x='Sulfate',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Conductivity'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Conductivity',ax=ax[1])
sns.histplot(data=data,x='Conductivity',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Organic_carbon'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Organic_carbon',ax=ax[1])
sns.histplot(data=data,x='Organic_carbon',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Trihalomethanes'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Trihalomethanes',ax=ax[1])
sns.histplot(data=data,x='Trihalomethanes',ax=ax[2],hue='Potability',kde=True)

In [ ]:
f,ax=plt.subplots(ncols=3,figsize=(20,6))
sns.distplot(data['Turbidity'],ax=ax[0])
sns.boxplot(data=data,x='Potability',y='Turbidity',ax=ax[1])
sns.histplot(data=data,x='Turbidity',ax=ax[2],hue='Potability',kde=True)

#### we can see that for every variable mean value corresponding to both the category (0,1) is same

### FEATURE SCALING

In [ ]:
X=data.drop(columns='Potability')
Y=data[['Potability']]

In [ ]:
sc=StandardScaler()

In [ ]:
X_scaled=sc.fit_transform(X)
X_scaled=pd.DataFrame(X_scaled,columns=X.columns)
X=X_scaled

### MODEL SELECTION

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.3,random_state=0)

In [ ]:
logreg=Pipeline([('logistic',LogisticRegression())])
decision=Pipeline([('dt',DecisionTreeClassifier())])
randomforest=Pipeline([('rf',RandomForestClassifier())])
naivebayes=Pipeline([('nb',GaussianNB())])
knn=Pipeline([('knn',KNeighborsClassifier())])
gbc=Pipeline([('gbc',GradientBoostingClassifier())])
adaboost=Pipeline([('adaboost',AdaBoostClassifier())])
sgdclassifier=Pipeline([('SGDclassifier',SGDClassifier())])
svc=Pipeline([('svc',SVC())])
mlpclass=Pipeline([('mlpc',MLPClassifier())])

In [ ]:
mypipeline1=[logreg,decision,randomforest,naivebayes,knn,gbc,adaboost,sgdclassifier,svc,mlpclass]

In [ ]:
pipelinedict1={0:'logistic',1:'dt',2:'rf',3:'nb',4:'knn',5:'gbc',6:'adaboost',7:'SGDclassifier',
              8:'svc',9:'mlpc'}

In [ ]:
for i in mypipeline1:
    i.fit(x_train,y_train)
model_score=[]
model_name=[]
for i,j in enumerate(mypipeline1):
    print("{} test accuracy: {}".format(pipelinedict1[i],j.score(x_test,y_test)))
    model_name.append(pipelinedict1[i])
    model_score.append(j.score(x_test,y_test))

In [ ]:
plt.barh(model_name,model_score)

##### we are getting the highest accuracy in SVC  model , so we choose this model for final model building

In [ ]:
svc1=SVC()

In [ ]:
svc1.fit(x_train,y_train)

In [ ]:
svc1.score(x_test,y_test)

### HYPERPARAMETER TUNING

In [ ]:
svc2=SVC()

In [ ]:
rand_state=[]
accuracy=[]
for i in range(1,150,1):
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=i)
    svc2.fit(x_train,y_train)
    accuracy.append(svc2.score(x_test,y_test))
    rand_state.append(i)

In [ ]:
plt.plot(rand_state,accuracy)

In [ ]:
for i in range(len(rand_state)):
    print(accuracy[i],rand_state[i])

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=91)

In [ ]:
svc3=SVC()

In [ ]:
svc3.fit(x_train,y_train)

In [ ]:
svc3.score(x_test,y_test)

In [ ]:
y_test_pred=svc3.predict(x_test)

In [ ]:
accuracy_score(y_test,y_test_pred)

#### we can see that accuracy of our svc model is become 71%

In [ ]:
plot_roc_curve(svc3,x_test,y_test)

In [ ]:
roc_auc_score(y_test,y_test_pred)

In [ ]:
plot_precision_recall_curve(svc3,x_test,y_test)

In [ ]:
print(classification_report(y_test,y_test_pred))